In [ ]:
import pandas as pd
from pathlib import Path
from autumn.projects.sm_covid2.common_school.calibration import get_bcm_object
from estival.sampling import tools as esamp
import yaml
from autumn.projects.sm_covid2.common_school.runner_tools import INCLUDED_COUNTRIES
import pickle

full_iso3_list = list(INCLUDED_COUNTRIES['all'].keys())

In [ ]:
full_iso3_list[49]

In [ ]:
analysis_folders = {
    "main": Path.cwd() / "31747883_full_analysis_26Sep2023_main",
    "increased_hh_contacts": Path.cwd() /"31902886_full_analysis_05Oct2023_increased_hh_contacts",
    "no_google_mobility": Path.cwd() /"31915437_full_analysis_05Oct2023_no_google_mobility"
}

In [ ]:
def get_mle_derived_outputs(iso3, analysis, save=True):
    best_params_path = analysis_folders[analysis] / iso3 / "retained_best_params.yml"
    with open(best_params_path, "r") as f:
        best_params = yaml.unsafe_load(f)
    mle_params = best_params[0]
    
    derived_outputs = {}
    for sc in ["baseline", "scenario_1"]:
        bcm = get_bcm_object(iso3, analysis=analysis, scenario=sc)
        res = esamp.model_results_for_samples([mle_params], bcm)
        derived_outputs[sc] = res.results.xs(0, level="sample", axis=1)

    if save:
        with open(analysis_folders[analysis] / iso3 / "derived_outputs.pickle", "wb") as f:
            pickle.dump(derived_outputs, f)

    return derived_outputs

In [ ]:
for analysis in analysis_folders:
    for iso3 in full_iso3_list:
        print(f"{analysis}: {iso3}")
        derived_outputs = get_mle_derived_outputs(iso3, analysis)